**Grid World**

In this exercise, we are going to use the former implementations that you should have already done in the previous exercises, in order to solve the next Grid World problem:

![alt text](grid_world.png "Title")

Consider a simple grid-world problem, where the environment is a rectangular grid. The state of the environment is given by the location of a robot in the grid. Hence, the cells of the grid correspond to the possible states of the environment. At each cell, four actions are possible: north, south, east and west, which deterministically (i.e., with probability 1) cause the agent to move one cell in the respective direction of the grid. Actions that would take the robot off the grid leave its location unchanged, but also results in a negative (undesired) reward of -1. Other actions result in a reward of 0, except those that move the agent out of the special states A and B. From state A, all four actions yield a positive (desired) reward of +10 and take the agent to A'. From state B, all four actions yield a positive reward of +5 and take the agent to B'.

Let us start with the imports. We use only numpy and matplotlib in this exercise.

In [14]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Now, let us define the parameters of the problem. We have an MDP with two states, and two actions. The rewards, discount factor, and transition probabilities are given in the figure above: let us explicitly write them (which is tedious).

In [15]:
gamma = 0.9
n_states = 25
n_actions = 4

def state_action_to_index(s, a): # Ancillary function to convert state-action pairs to indexes
    if isinstance(a, str):
        a = ['N', 'S', 'E', 'W'].index(a)  # Mapping of actions to indexes
    return s * n_actions + a

def index_to_state_action(i): # Ancillary function to convert indexes to state-action pairs
    return i // n_actions, i % n_actions

P = np.zeros((n_states * n_actions, n_states))  # We need to fill the non-zero values, which depend on each state
R = np.zeros((n_states * n_actions, 1))  # Already filled with zeros, we only need to add the non-zero values

# We fill the non-zero values of P and R case by case:
for s in range(n_states):
    if s == 0: # Left up corner
        P[state_action_to_index(s, 'N'), 0] = P[state_action_to_index(s, 'S'), 5] = P[state_action_to_index(s, 'E'), 1] = P[state_action_to_index(s, 'W'), 0] = 1
        R[state_action_to_index(s, 'N')] = R[state_action_to_index(s, 'W')] = -1
    elif s == 1:  # Special state A
        P[state_action_to_index(s, 'N'), 21] = P[state_action_to_index(s, 'S'), 21] = P[state_action_to_index(s, 'E'), 21] = P[state_action_to_index(s, 'W'), 21] = 1
        R[state_action_to_index(s, 'N')] = R[state_action_to_index(s, 'S')] = R[state_action_to_index(s, 'E')] = R[state_action_to_index(s, 'W')] = 10
    elif s == 2:
        P[state_action_to_index(s, 'N'), 2] = P[state_action_to_index(s, 'S'), 7] = P[state_action_to_index(s, 'E'), 1] = P[state_action_to_index(s, 'W'), 3] = 1
        R[state_action_to_index(s, 'N')] = -1
    elif s == 3: # Special state B
        P[state_action_to_index(s, 'N'), 13] = P[state_action_to_index(s, 'S'), 13] = P[state_action_to_index(s, 'E'), 13] = P[state_action_to_index(s, 'W'), 13] = 1
        R[state_action_to_index(s, 'N')] = R[state_action_to_index(s, 'S')] = R[state_action_to_index(s, 'E')] = R[state_action_to_index(s, 'W')] = 5
    elif s == 4:  # Right up corner
        P[state_action_to_index(s, 'N'), s] = P[state_action_to_index(s, 'S'), s + 5] = P[state_action_to_index(s, 'E'), s] = P[state_action_to_index(s, 'W'), s - 1] = 1
        R[state_action_to_index(s, 'N')] = R[state_action_to_index(s, 'E')] = -1
    elif s == 20:  # Lower Left corner
        P[state_action_to_index(s, 'N'), s - 5] = P[state_action_to_index(s, 'S'), s] = P[state_action_to_index(s, 'E'), s + 1] = P[state_action_to_index(s, 'W'), s] = 1
        R[state_action_to_index(s, 'S')] = R[state_action_to_index(s, 'W')] = -1
    elif s == 24:  # Lower Right corner
        P[state_action_to_index(s, 'N'), s - 5] = P[state_action_to_index(s, 'S'), s] = P[state_action_to_index(s, 'E'), s] = P[state_action_to_index(s, 'W'), s - 1] = 1
        R[state_action_to_index(s, 'S')] = R[state_action_to_index(s, 'E')] = -1
    elif s == 5 or s == 10 or s == 15:  # Left margin
        P[state_action_to_index(s, 'N'), s - 5] = P[state_action_to_index(s, 'S'), s + 5] = P[state_action_to_index(s, 'E'), s + 1] = P[state_action_to_index(s, 'W'), s] = 1
        R[state_action_to_index(s, 'W')] = -1
    elif s == 9 or s == 14 or s == 19:  # Right margin
        P[state_action_to_index(s, 'N'), s - 5] = P[state_action_to_index(s, 'S'), s + 5] = P[state_action_to_index(s, 'E'), s] = P[state_action_to_index(s, 'W'), s - 1] = 1
        R[state_action_to_index(s, 'E')] = -1
    elif s == 21 or s == 22 or s == 23:  # Down margin
        P[state_action_to_index(s, 'N'), s - 5] = P[state_action_to_index(s, 'S'), s] = P[state_action_to_index(s, 'E'), s + 1] = P[state_action_to_index(s, 'W'), s - 1] = 1
        R[state_action_to_index(s, 'S')] = -1
    else:  # Center cells
        P[state_action_to_index(s, 'N'), s - 5] = P[state_action_to_index(s, 'S'), s + 5] = P[state_action_to_index(s, 'E'), s + 1] = P[state_action_to_index(s, 'W'), s - 1] = 1

Now, reuse the implementations from the previous exercises to define the methods we are going to use: Policy Evaluation (using the Bellman equations and the iterative method), Policy Iteration, and Value Iteration. For simplicity, we will only use in thie exercise the state value function (although the same results would be obtained with the state-action value function).

In [16]:
def Bellman_PE(pi): # pi is the policy to evaluate

    # To be filled by the student

    return v_opt.flatten()  # Return the value function

def Policy_Evaluation(pi):  # pi is the policy to evaluate

    # To be filled by the student

    return v_pi.flatten()  # Return the value function

def Policy_Iteration(pi_init):  # pi init is the initial policy

    # To be filled by the student

    return pi_pe.flatten(), v_pi_pe.flatten()  # Return the optimal policy and the associated value function

def Value_Iteration(): # No arguments for Value Iteration

    # To be filled by the student

    return pi_vi.flatten(), v_vi.flatten()  # Return the optimal policy and the associated value function

First, let us assume a Uniform Random policy: let us obtain the value function for this policy using the Bellman equations and the iterative method:

In [17]:
pi_random = np.zeros((n_states, n_states * n_actions))
for s in range(n_states):
    pi_random[s, s * n_actions: (s + 1) * n_actions] = 1 / n_actions  # This is a uniform random policy

v_bellman = Bellman_PE(pi_random)
v_pe = Policy_Evaluation(pi_random)

with np.printoptions(precision=2, suppress=True):  # Print the values obtained
    print("v^* theory")
    print(v_bellman.reshape((5, 5)))
    print("v^* PI")
    print(v_pe.reshape((5, 5)))

v^* theory
[[ 3.31  8.79  4.43  5.32  1.49]
 [ 1.52  2.99  2.25  1.91  0.55]
 [ 0.05  0.74  0.67  0.36 -0.4 ]
 [-0.97 -0.44 -0.35 -0.59 -1.18]
 [-1.86 -1.35 -1.23 -1.42 -1.98]]
v^* PI
[[ 3.32  8.8   4.43  5.33  1.5 ]
 [ 1.53  3.    2.26  1.91  0.55]
 [ 0.06  0.74  0.68  0.36 -0.4 ]
 [-0.97 -0.43 -0.35 -0.58 -1.18]
 [-1.85 -1.34 -1.22 -1.42 -1.97]]


Second, we are going to obtain the optimal policy and the associated value function using both Policy Iteration and Value Iteration:

In [18]:
pi_opt, v_opt = Policy_Iteration(pi_random)
pi_vi, v_vi = Value_Iteration()

with np.printoptions(precision=2, suppress=True):  # Print the values obtained
    print("v^* PI")
    print(v_opt.reshape((5, 5)))
    print("v^* VI")
    print(v_vi.reshape((5, 5)))

v^* PI
[[21.98 24.42 21.98 19.42 17.48]
 [19.78 21.98 19.78 17.8  16.02]
 [17.8  19.78 17.8  16.02 14.42]
 [16.02 17.8  16.02 14.42 12.98]
 [14.42 16.02 14.42 12.98 11.68]]
v^* VI
[[21.98 24.42 21.98 19.42 17.48]
 [19.78 21.98 19.78 17.8  16.02]
 [17.8  19.78 17.8  16.02 14.42]
 [16.02 17.8  16.02 14.42 12.98]
 [14.42 16.02 14.42 12.98 11.68]]
